質問文を学習させ、正当な分類を行う（Answer_output.ipynb の続き）

In [1]:
import pandas as pd
# データフレームの取得
file_path = "C:/Users/iput Hata Sakuto/cuda-11.8/提出/自作モデル-1/chunk2question_after.xlsx"
df = pd.read_excel(file_path)
df

,Unnamed: 0.1,Unnamed: 0,chunk,label_num,label
0,0,0,現代社会において、多様な要素が複雑に絡み合い、常に変化している課題について、どのようなアプロ...,0,専門職大学
1,1,1,現代社会の課題に対応するためには、どのような知識やスキルが必要でしょうか？,0,専門職大学
2,2,2,現代社会の課題を解決するには、単一の専門分野の知識だけでは不十分ですか？,0,専門職大学
3,3,3,現代社会の課題に対応するには、単一の専門知だけではなく、多様な要素を考慮し常に変化する知識＜...,0,専門職大学
4,4,4,現代社会の課題解決には、どのような知識やスキルが必要だと考えられますか？,0,専門職大学
...,...,...,...,...,...
1275,1275,1275,深層学習のニューラルネットワークの技術内容と実現手法、効果について詳しく学び、具体的な適用方...,3,授業
1276,1276,1276,深層学習のニューラルネットワーク技術について、具体的な実装例を通じて学びますか？また、畳み込...,3,授業
1277,1277,1277,深層学習の主要技術であるニューラルネットワークの技術内容や実現手法、効果はどのようなものです...,3,授業
1278,1278,1278,深層学習の主要技術であるニューラルネットワークの技術内容と実現手法、効果について具体的な適用...,3,授業


TensorFlow2: Universal Sentence Encoder

In [4]:
from tensorflow_hub import load
import numpy as np

# Universal Sentence Encoderをロード
model = load("https://tfhub.dev/google/universal-sentence-encoder/4")

# df["chunk"]のテキストデータをベクトル化
df["vector_USE4"] = df["chunk"].apply(lambda x: model(np.array([str(x)]))[0].numpy())
df

,Unnamed: 0.1,Unnamed: 0,chunk,label_num,label,vector_USE4
0,0,0,現代社会において、多様な要素が複雑に絡み合い、常に変化している課題について、どのようなアプロ...,0,専門職大学,"[-0.04152658, -0.0011149249, 0.017183743, 0.00..."
1,1,1,現代社会の課題に対応するためには、どのような知識やスキルが必要でしょうか？,0,専門職大学,"[0.019241601, 0.030672591, 0.048391387, 0.0366..."
2,2,2,現代社会の課題を解決するには、単一の専門分野の知識だけでは不十分ですか？,0,専門職大学,"[0.015363986, -0.025413968, 0.06662946, 0.0339..."
3,3,3,現代社会の課題に対応するには、単一の専門知だけではなく、多様な要素を考慮し常に変化する知識＜...,0,専門職大学,"[0.062338244, -0.03355772, 0.03556679, 0.04975..."
4,4,4,現代社会の課題解決には、どのような知識やスキルが必要だと考えられますか？,0,専門職大学,"[0.047757354, -0.07123919, 0.04704295, 0.02480..."
...,...,...,...,...,...,...
1275,1275,1275,深層学習のニューラルネットワークの技術内容と実現手法、効果について詳しく学び、具体的な適用方...,3,授業,"[-0.039543577, -0.022720978, 0.040409163, 0.04..."
1276,1276,1276,深層学習のニューラルネットワーク技術について、具体的な実装例を通じて学びますか？また、畳み込...,3,授業,"[0.037995122, 0.043231163, 0.022386065, 0.0307..."
1277,1277,1277,深層学習の主要技術であるニューラルネットワークの技術内容や実現手法、効果はどのようなものです...,3,授業,"[0.032299045, -0.027745942, 0.0033454273, 0.00..."
1278,1278,1278,深層学習の主要技術であるニューラルネットワークの技術内容と実現手法、効果について具体的な適用...,3,授業,"[-0.014120439, -0.01623691, 0.022620294, 0.079..."


モデル作成

In [6]:
import pandas as pd
import tensorflow_hub as hub
# import numpy as np
import tensorflow_text
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# 訓練データと評価データに分割する
df_train, df_test = train_test_split(df, test_size=0.2, stratify=df.label)
df_train = df_train.copy()
df_test = df_test.copy()
df_train.reset_index(inplace=True, drop=True)
df_test.reset_index(inplace=True, drop=True)

# USEモデルの読み込みと、テキストのベクトル化
url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
embed = hub.load(url)

X_train = embed(df_train.chunk).numpy()
X_test = embed(df_test.chunk).numpy()

# 正解ラベル(記事カテゴリ)を One-Hot 表現に変換

ohe = OneHotEncoder()
ohe.fit(df_train.label.values.reshape(-1, 1))
y_train = ohe.transform(df_train.label.values.reshape(-1, 1)).toarray()
y_test = ohe.transform(df_test.label.values.reshape(-1, 1)).toarray()

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# モデルの作成
model = Sequential()
model.add(Input(shape=(512, )))
model.add(Dropout(0.3))
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.4))
model.add(Dense(32, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))
print(model.summary())

model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=['acc']
)

# 学習
early_stopping = EarlyStopping(
                        monitor='val_loss',
                        min_delta=0.0,
                        patience=5,
                )

history = model.fit(X_train, y_train,
                    batch_size=128,
                    epochs=1000,
                    verbose=2,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping],
                    )

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 128)               65664     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 32)                4128      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 4)                 132       
                                                                 
Total params: 69,924
Trainable params: 69,924
Non-traina

In [14]:
import tensorflow as tf

# 別のデータでクラスラベルを予測
url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
embed = hub.load(url)

new_data = ["専門職大学について","専門職大学の実習について","専門職大学の特徴について"]
# 期待するラベル = [0,2,1]

new_target_data = embed(new_data).numpy()
X_new = new_target_data

# モデルに新しいデータを渡して予測
predictions = model.predict(X_new)

# 予測結果を出力
print("Raw predictions:")
print(predictions)

# 各データポイントにおいて最も高い確率のクラスを取得
predicted_labels = tf.argmax(predictions, axis=1).numpy()

# 予測されたクラスラベルを出力
print("Predicted labels:")
print(predicted_labels)

Raw predictions:
[[3.29497107e-03 7.72940159e-01 5.15159452e-04 2.23249719e-01]
 [2.91403264e-01 1.08787954e-01 3.53808136e-04 5.99454999e-01]
 [9.32329427e-03 1.55147329e-01 2.44968059e-03 8.33079636e-01]]
Predicted labels:
[1 3 3]


# ＜結果＞
モデル自体の精度は一見高く見えたが、実際の質問文類にて結果が振るわなかった。